# Resume parsing using Spacy



In [19]:
import spacy
import pickle
import random

In [2]:
train_data = pickle.load(open('train_data.pkl', 'rb'))

In [4]:
train_data[0]

('Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [3]:
train_data[0][1].get('entities')

[(1749, 1755, 'Companies worked at'),
 (1696, 1702, 'Companies worked at'),
 (1417, 1423, 'Companies worked at'),
 (1356, 1793, 'Skills'),
 (1209, 1215, 'Companies worked at'),
 (1136, 1248, 'Skills'),
 (928, 932, 'Graduation Year'),
 (858, 889, 'College Name'),
 (821, 856, 'Degree'),
 (787, 791, 'Graduation Year'),
 (744, 750, 'Companies worked at'),
 (722, 742, 'Designation'),
 (658, 664, 'Companies worked at'),
 (640, 656, 'Designation'),
 (574, 580, 'Companies worked at'),
 (555, 573, 'Designation'),
 (470, 493, 'Companies worked at'),
 (444, 469, 'Designation'),
 (308, 314, 'Companies worked at'),
 (234, 240, 'Companies worked at'),
 (175, 198, 'Companies worked at'),
 (93, 137, 'Email Address'),
 (39, 48, 'Location'),
 (13, 38, 'Designation'),
 (0, 12, 'Name')]

# Training the Model

https://spacy.io/usage/training#training-data   // code to train data using spacy

###  Named Entity Recognition with Spacy 

###  create an empty model using spacy.blank with the ID of your language

In [7]:
nlp = spacy.blank('en') 

### Add the new entity label to the entity recognizer using the add_label method. You can access the entity recognizer in the pipeline via nlp.get_pipe('ner')

In [8]:
def train_model(train_data):
    if "ner" not in nlp.pipe_names:  # if ner is not in pipeline, add it to pipelinr
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)  # add ner at last
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")
        
    # add labels
    for _, annotations in train_data:  # we have text and annotations in train data
        for ent in annotations.get("entities"): # from annotation, we get the entities
            ner.add_label(ent[2])  #add its label inside the ner . see above. 2nd position is label
    
    #--------------------------
    
    # get names of other pipes to disable them during training
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
   
    # only train NER
    with nlp.disable_pipes(*other_pipes), warnings.catch_warnings():
        # show warnings for misaligned entity spans once
        warnings.filterwarnings("once", category=UserWarning, module='spacy')

        # reset and initialize the weights randomly – but only if we're
        # training a new model
        optimizer = nlp.begin_training()
            
        for itn in range(20): # no of iterations is 20
            print('Starting iterations ', str(itn))
            random.shuffle(train_data)
            
            
            losses = {}
            index = 0
            for text, annotations in train_data:
                print(index)
                index = index + 1
                try:
                    nlp.update(
                        [text],  # batch of texts
                        [annotations],  # batch of annotations
                        drop=0.2,  # dropout - make it harder to memorise data
                        sgd=optimizer,  # callable to update weights
                        losses=losses)
                except Exception as e:
                    pass
#                     print(text)
                
            print(losses)

### nlp.update : Loop over the examples and call nlp.update, which steps through the words of the input. At each word, it makes a prediction. It then consults the annotations, to see whether it was right. If it was wrong, it adjusts its weights so that the correct action will score higher next time.


In [10]:
import warnings

In [11]:
train_model(train_data)

Starting iterations  0
0


C:\Users\Vikee\anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Pratik Vaidya Pune, Maharashtra - Email me on Inde..." with entities "[(1901, 1906, 'Skills'), (1812, 1816, 'Graduation ...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)


1
2
3


C:\Users\Vikee\anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Jacob Philip Kottayam, Kerala - Email me on Indeed..." with entities "[(2165, 2215, 'Skills'), (2145, 2155, 'Degree'), (...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)


4


...

C:\Users\Vikee\anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Jatin Arora SDET Automation Engineer, Infosys - CR..." with entities "[(3909, 3931, 'College Name'), (3883, 3908, 'Degre...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\Vikee\anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Anil Kumar Microsoft Azure (Basic Management)  Del..." with entities "[(8004, 8009, 'Location'), (7716, 7721, 'Location'...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)


198


C:\Users\Vikee\anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Prasanna Ignatius MICROSOFT - Backup Administrator..." with entities "[(3917, 3975, 'Skills'), (3790, 3835, 'College Nam...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)


199
{'ner': 13820.04584470513}
Starting iterations  1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
{'ner': 8538.699470382337}
Starting iterations  2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71


3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
{'ner': 3343.62019791448}
Starting iterations  13
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91


### Save the trained model using nlp.to_disk

In [13]:
nlp.to_disk('nlp_model')  # store the model to use it later without training again

In [14]:
# open the nlp_model , open meta.json. it has the labels

# Testing the Model

Test the model to make sure the new entity is recognized correctly.

In [15]:
nlp_model = spacy.load('nlp_model')

In [17]:
text = train_data[0][0]
text  # whole text data

"Rahul Tayade Global Production Support Lead, - Infosys Ltd (Technology Lead) - HSBC  Pune, Maharashtra - Email me on Indeed: indeed.com/r/Rahul-Tayade/ce40c3731cb69763  • Total 12+ years of IT experience in the analysis, design, development, Testing support, implementation, CAT support and management of full life cycle applications, project coordination, managing development and support projects. • More than 7 years of experience on Project Leading support and maintenance project including enhancements of the application. • Over 4 years of Project Management experience on support/maintenance projects. • Managed multiple applications with team more than 16 members. • Involved in PM activities like dealing with customer, identifying new business, get new business, accordingly raising quotes, and get PO approved from customer, track the work and bill customer based on Resource Utilization, Workforce management, manage project costing by providing cost-effective solutions etc. • Involve i

In [ ]:
# Make predictions on text data

In [18]:
doc = nlp_model(text)
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}') # 30 character padding

NAME                          - Rahul Tayade
COMPANIES WORKED AT           - Infosys Ltd (Technology Lead) - HSBC
LOCATION                      - Pune
EMAIL ADDRESS                 - indeed.com/r/Rahul-Tayade/ce40c3731cb69763
COMPANIES WORKED AT           - Infosys Ltd (Technology Lead)
LOCATION                      - Pune
LOCATION                      - Pune
DESIGNATION                   - Project Lead
DEGREE                        - British Telecom UK
LOCATION                      - Pune
DESIGNATION                   - System Analyst
DEGREE                        - Bachelor of Engineering in ELECTRONICS AND TELECOMMUNICATION
COLLEGE NAME                  - Amravati University
LOCATION                      - Amravati
